# Predictive Modeling and Evaluation -  Dataset 1

In this notebook, we will build and evaluate predictive models using **Dataset 1**.  
The main goals are:  

- Prepare the data for machine learning (features, target, train/test split, scaling).  
- Train a set of baseline and advanced models (e.g., Logistic Regression, Decision Tree, Random Forest, KNN, SVM).  
- Evaluate model performance using metrics such as Accuracy, Precision, Recall, F1-score, and ROC-AUC.  
- Apply cross-validation to ensure robust results.  
- Analyze feature importance and model interpretability.  
- Select the most promising model(s) for further comparison with Dataset 2.  


In [5]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import shapiro, mannwhitneyu

from src.overview_functions import convert_to_string_to_lower, display_column_distribution, display_scatterplot_by_features
from src.stats_tests_functions import multiple_results_mwu

In [6]:
data = pd.read_csv('../data/cleaned_data/dataset1_cleaned.csv')
data

,age,gender,heart_rate,pressure_high,pressure_low,glucose,kcm,troponin,heart_disease
0,64,1,66,160,83,160.0,1.80,0.012,0
1,21,1,94,98,46,296.0,6.75,1.060,1
2,55,1,64,160,77,270.0,1.99,0.003,0
3,64,1,70,120,55,270.0,13.87,0.122,1
4,55,1,64,112,65,300.0,1.08,0.003,0
...,...,...,...,...,...,...,...,...,...
1311,44,1,94,122,67,204.0,1.63,0.006,0
1312,66,1,84,125,55,149.0,1.33,0.172,1
1313,45,1,85,168,104,96.0,1.24,4.250,1
1314,54,1,58,117,68,443.0,5.80,0.359,1
